## Installing the libraries


The Notebook that you're currently viewing in the Thinkful curriculum reader has the two required libraries already installed. But if you need to create a Notebook on Colab that requires these libraries, you'll need to install them manually. To do that, create and run a cell with the following commands:

```
!pip install sqlalchemy
!pip install psycopg2
!pip install psycopg2-binary
```

After these libraries install, you'll be able to import them and use them in your Python code.

In [1]:
# Import the SQLAlchemy engine
from sqlalchemy import create_engine

The import statement shown above makes the `create_engine()` function available in your program. You can use that function to create a connection to the database. The database will reside on a database server and have specific authentication and permissions set. Recall what you learned earlier in this program about connecting to PostgreSQL servers with a username. Your Python code also needs some information in order to connect. Specifically, you need the following:

* **The username**
* **A password**
* **The hostname:** The domain name of a server or an IP address
* **A port number:** By default, the port number of PostgreSQL database servers is 5432. But in some cases, the server administrator may decide to use a different port. In such cases, you need to specify the port number.
* **A database name:** Each server may host multiple databases. When you connect, you must state which database you wish to connect to.

You can declare some variables to store these values, as demonstrated below. Then use the `create_engine()` function to connect to the database.

In [2]:
# Database credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'medicalcosts'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

/opt/conda/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Insurance charges dataset

You will explore database connections and simple queries on a medical cost dataset that's originally from Kaggle. You'll be working with a SQL-based version of this dataset hosted on a Postgres server that's accessible via the credentials given above.

The data is about the costs charged by insurance companies to individuals. These are costs that the insured individuals were required to pay after the insurance company made payments to medical providers. The dataset includes seven variables, as described below:

* `age`: The age of the individual. It's a numeric variable.
* `sex`: The biological sex of the insured. It is a categorical variable with two values: `male` and `female`.
* `bmi`: Body mass index. This index indicates whether an individual's weight is relatively high or low, based on height. This index defines an ideal range between `18.5` and `24.9`. The variable is continuous.
* `children`: How many dependent children the insured person has. Dependents are covered by the insurance plan. This is a numeric variable.
* `smoker`: This categorical variable (`yes` or `no`) indicates whether or not the person smokes.
* `region`: The person's residential area in the US. It's a categorical variable with four values: `northeast`, `southeast`, `southwest`, and `northwest`).
* `charges`: The medical costs (in US dollars) billed by the insurance company. It's a continuous variable.

This database has a single table, *medicalcosts*, with these seven columns. Below, quickly check to see if you can query the table and get the data from the table.

In [3]:
# Use the connection to execute some SQL
insurance = engine.execute('SELECT * FROM medicalcosts LIMIT 10')

# No need for an open connection, because you're only doing a single query
engine.dispose()

# Process the results
for row in insurance:
  print(row)


(19, 'female', 27.9, 0, 'yes', 'southwest', 16884.9)
(18, 'male', 33.77, 1, 'no', 'southeast', 1725.55)
(28, 'male', 33.0, 3, 'no', 'southeast', 4449.46)
(33, 'male', 22.705, 0, 'no', 'northwest', 21984.5)
(32, 'male', 28.88, 0, 'no', 'northwest', 3866.86)
(31, 'female', 25.74, 0, 'no', 'southeast', 3756.62)
(46, 'female', 33.44, 1, 'no', 'southeast', 8240.59)
(37, 'female', 27.74, 3, 'no', 'northwest', 7281.51)
(37, 'male', 29.83, 2, 'no', 'northeast', 6406.41)
(60, 'female', 25.84, 0, 'no', 'northwest', 28923.1)


## The `ResultProxy` object

In the above code, the `execute()` method of the connection was used to execute some raw SQL on the database. As mentioned above, SQLAlchemy has many features beyond this, but executing these SQL statements is all that you need for now. The result of executing a query with the `execute()` method is a `ResultProxy` object. `ResultProxy` provides several methods and properties that you can use to process the data returned from the query.

In the example above, you simply iterated over all the rows of data that were returned. Next, examine some of the features:

In [4]:
# Get a list of the keys (column names) 
insurance.keys()

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

The `ResultProxy` object exposes the rows as an *iterator*. That means that you have a single pass at the rows, but accessing them again isn't possible. If you try running the `for` loop again, you will notice that you get no output. Take a look:

In [5]:
for row in insurance:
  print(row)

In many instances, this isn't a problem; you may only need to iterate the rows once to process them. However, if you need to perform several iterations of the results, then it would be better to take the rows from the `ResultProxy` and construct a *list* of rows, as shown here:

In [6]:
# Execute the SQL statement again
insurance = engine.execute('SELECT * FROM medicalcosts LIMIT 10')

# Dispose the connection
engine.dispose()

# Use `fetchall()` to get a list of rows from the results
rows = insurance.fetchall()

# Now, process the list of rows
for row in rows:
  print(row)

  

(19, 'female', 27.9, 0, 'yes', 'southwest', 16884.9)
(18, 'male', 33.77, 1, 'no', 'southeast', 1725.55)
(28, 'male', 33.0, 3, 'no', 'southeast', 4449.46)
(33, 'male', 22.705, 0, 'no', 'northwest', 21984.5)
(32, 'male', 28.88, 0, 'no', 'northwest', 3866.86)
(31, 'female', 25.74, 0, 'no', 'southeast', 3756.62)
(46, 'female', 33.44, 1, 'no', 'southeast', 8240.59)
(37, 'female', 27.74, 3, 'no', 'northwest', 7281.51)
(37, 'male', 29.83, 2, 'no', 'northeast', 6406.41)
(60, 'female', 25.84, 0, 'no', 'northwest', 28923.1)


Now, if you repeat the `for` loop or try to access the rows in the list again, they will still be available. Additionally, you can use the features of the Python list data structure to access individual rows or groups of rows. For example, to just iterate the first five rows, you can slice the list, like this:

In [7]:
for row in rows[:5]:
  print(row)

(19, 'female', 27.9, 0, 'yes', 'southwest', 16884.9)
(18, 'male', 33.77, 1, 'no', 'southeast', 1725.55)
(28, 'male', 33.0, 3, 'no', 'southeast', 4449.46)
(33, 'male', 22.705, 0, 'no', 'northwest', 21984.5)
(32, 'male', 28.88, 0, 'no', 'northwest', 3866.86)


Or you can get the last five rows, like this:

In [8]:
for row in rows[-5:]:
  print(row)

(31, 'female', 25.74, 0, 'no', 'southeast', 3756.62)
(46, 'female', 33.44, 1, 'no', 'southeast', 8240.59)
(37, 'female', 27.74, 3, 'no', 'northwest', 7281.51)
(37, 'male', 29.83, 2, 'no', 'northeast', 6406.41)
(60, 'female', 25.84, 0, 'no', 'northwest', 28923.1)


## The `RowProxy` object

You will look at a few more examples of manipulating the list shortly. But first, take a closer look at the row itself. When you print the row, you get a tuple of values like this:

```
(18, 'female', 31.92, 0, 'no', 'northeast', 2205.98)
```

But what exactly is that? SQLAlchemy constructs the `ResultProxy` out of a set of `RowProxy` objects. `RowProxy` provides several methods for using the data in a row. You can always check the type of an object by using the `type()` function.

In [9]:
# Get the first row from the list
first_row = rows[0]

# Get the data type of this row
type(first_row)

sqlalchemy.engine.result.RowProxy

#### The `in` operator

The `in` operator returns `True` if a given key exists in the row, and `False` otherwise. This can be used to verify that a particular data value exists before you process the row.

In [10]:
print('The first row has a key named `age`: {}'.format('age' in first_row))
print('The first row has a key named `height`: {}'.format('height' in first_row))

The first row has a key named `age`: True
The first row has a key named `height`: False


#### The `keys()` method
The `keys()` method lists all the keys in the row, similar to the `keys()` method of the `ResultProxy` itself.

In [11]:
first_row.keys()

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

#### The `items()` method

The `items()` method returns a list of tuples containing the key-value pairs from the row.

In [12]:
first_row.items()

[('age', 19),
 ('sex', 'female'),
 ('bmi', 27.9),
 ('children', 0),
 ('smoker', 'yes'),
 ('region', 'southwest'),
 ('charges', 16884.9)]

This has implications for how the data is accessed as you process a row. There are two different ways to get the value of a specific column in a row. The first column is at position 0, the second is at position 1, and so on, and you can use the numeric position of the column in the result. However, depending on the position can be unstable, because a database table doesn't guarantee the order of columns. So if the SQL statement changes a little, then all positions could be wrong. So it's safer to use the actual column name instead. Here are examples of both methods:



In [13]:
# Access it by position
bmi = first_row[2]

# Access it by column name
charges = first_row['charges']

print('The first row has a bmi of {} and charges of {}'.format(bmi, charges))

The first row has a bmi of 27.9 and charges of 16884.9


## Processing the data

At this point, you know how to execute a query against a database and access the returned data. Now, walk through a quick example of processing the data using the same medical cost database as above. You can calculate basic statistics like mean, median, range, and variance in Python. Note that there are many libraries available that can perform these calculations for you. But it is instructive to see how you can do it yourself first, then look at the external libraries later.

Suppose your manager wanted a report with the mean, median, and standard deviation of `charges` for all females between the ages of 18 and 30. Start by working out the SQL statement.

### The `WHERE` clause

You need to restrict the rows that you process to only those where the `sex` field contains the value `female`, and the `age` field contains a value between `18` and `30`. That means that you have three conditions to add to the SQL statement. Also, the results don't actually need to include all seven fields—just `charges`. The SQL statement might look like this:

```sql
SELECT
  charges
FROM
  medicalcosts
WHERE
  sex = 'female'
  AND age BETWEEN 18 and 30;
```

Before you actually perform the calculations, take a look at the effect of this `WHERE` clause.


In [14]:
# Find the total number of rows in the table
row_count = engine.execute('SELECT COUNT(*) FROM medicalcosts')

# Dispose the connection
engine.dispose()

# Fetch the first row from the results
ans = row_count.first()

# Print the count
print('There are {} rows in the table'.format(ans['count']))

There are 1338 rows in the table


There are 1,338 rows in total. That includes both females and males, as well as females that aren't between the ages of 18 and 30. Now, try adding the filter conditions. Because the SQL statement is getting longer, you can use multi-line strings in Python to write a formatted SQL statement for better readability.

In [15]:
sql = '''
SELECT 
  COUNT(*) 
FROM 
  medicalcosts
WHERE 
  sex = 'female' '''

row_count = engine.execute(sql)
engine.dispose()
ans = row_count.first()
print('There are {} females in the table'.format(ans['count']))

There are 662 females in the table


That seems reasonable; approximately half of the data refers to females. Now, add the next condition:

In [16]:
sql = '''
SELECT 
  COUNT(*) 
FROM 
  medicalcosts
WHERE 
  sex = 'female'
  AND age BETWEEN 18 AND 30'''

row_count = engine.execute(sql)
engine.dispose()
ans = row_count.first()
print('There are {} females between 18 and 30 in the table'.format(ans['count']))

There are 214 females between 18 and 30 in the table


Checking like this isn't always necessary, but it helps you to confirm that the SQL statement is correct. You can also use your favorite SQL client to work out the correct SQL before using it in the Python code. Now, get the 214 values into a list and perform the calculations.

In [17]:
sql = '''
SELECT 
  charges 
FROM 
  medicalcosts
WHERE 
  sex = 'female'
  AND age BETWEEN 18 AND 30'''

charges = engine.execute(sql)
engine.dispose()

rows = charges.fetchall()

# At this point, the rows list contains `RowProxy` objects,
# but you really only need the numeric value. So extract
# the numeric value into a list of raw numbers:

charges = [x['charges'] for x in rows]
print(charges)

[16884.9, 4149.74, 2198.19, 4687.8, 3046.06, 3393.36, 3556.92, 2211.13, 3579.83, 4133.64, 14711.7, 1743.21, 16577.8, 3947.41, 2755.02, 2026.97, 3766.88, 21344.8, 2331.52, 2404.73, 19107.8, 2257.48, 3385.4, 17081.1, 2457.21, 2155.68, 2166.73, 5138.26, 36149.5, 4830.63, 2719.28, 2855.44, 1631.82, 3981.98, 5325.65, 4922.92, 2130.68, 37133.9, 4337.74, 3392.98, 25081.8, 1727.79, 1615.77, 38511.6, 17085.3, 2150.47, 3172.02, 2156.75, 4349.46, 20177.7, 4151.03, 1737.38, 34838.9, 24671.7, 3561.89, 18955.2, 3554.2, 14133.0, 1607.51, 13844.5, 2597.78, 3180.51, 18034.0, 15006.6, 3176.82, 4618.08, 2138.07, 1631.67, 14455.6, 2134.9, 7323.73, 3167.46, 18804.8, 4906.41, 19522.0, 23288.9, 2201.1, 2203.47, 1744.46, 1622.19, 1748.77, 2196.47, 3044.21, 11482.6, 24059.7, 3056.39, 3213.62, 17878.9, 2801.26, 2128.43, 2219.45, 4753.64, 3206.49, 15359.1, 1633.04, 17469.0, 3558.62, 2207.7, 1880.07, 34439.9, 3736.46, 3366.67, 2709.11, 4466.62, 3410.32, 3943.6, 2585.27, 3578.0, 3201.25, 3500.61, 2020.55, 2457.5, 

### Mean

Calculating the mean is fairly straightforward. First, you sum the values in the array. Then, you divide the sum by the number of items in the array. Because finding the mean of an array of numbers is something that you may do many times for many different arrays, you could write a function that performs that task for you, then invoke the function whenever you need to find the mean of an array. 

There are many ways that this code could be written. If you do a search online, you will probably find many different answers, including the use of more complex tools like lambda functions and functools. Those are all perfectly valid, and may even perform better than this example. But here, you are aiming for simple and readable.

In [18]:
# A simple implementation simply iterating the array
def mean(numbers):
  total_sum = 0;  # Initialize the sum to zero
  for n in numbers:
    total_sum += n # Add up the numbers in the array
  count = len(numbers)   # Find the length of the array
  avg = total_sum / count  # Calculate the mean
  return avg  # Return the result

print('The mean of the charges is {}'.format(mean(charges)))

The mean of the charges is 8624.224345794393


You can also leverage SQL queries to do the calculation for you. To do this, use the `AVG()` function for `charges`, as shown here:

In [19]:
sql = '''
SELECT 
  AVG(charges)
FROM 
  medicalcosts
WHERE 
  sex = 'female'
  AND age BETWEEN 18 AND 30'''


charges_avg = engine.execute(sql)
engine.dispose()

rows_avg = charges_avg.fetchall()[0][0] 

print('The mean of the charges is {}'.format(rows_avg))


The mean of the charges is 8624.2230966158


### Median

The median is the number occurring in the middle of the array when the array is sorted. So you can find the median by first sorting the array, then finding the number in the exact middle. If there is an odd number of values in the array, then finding the middle number is easy. But if there is an even number, there is no obvious middle. In that case, you find the two numbers closest to the middle and calculate their average.

Python has built in the ability to sort a list. The simplest approach is to use the `sort()` method of the list itself. It sorts the list in place, which means that the list itself is changed.

**Note:** Unfortunately, identifying the median in SQL is very challenging, so you are going to stick with Python code here.

In [20]:
alist = [5, 2, 4, 1, 3]
alist.sort()
print(alist)

[1, 2, 3, 4, 5]


If making changes to the list itself isn't desirable, then you can instead use the `sorted()` function, which creates a new sorted list. This second approach has the advantage of working on other data structures than lists, too. 

In [21]:
alist = [5, 2, 4, 1, 3]
sorted_list = sorted(alist)
print(sorted_list)
print(alist)

[1, 2, 3, 4, 5]
[5, 2, 4, 1, 3]


Notice how even though you end up with a sorted list named `sorted_list`, you still have the original list untouched. For the problem at hand, it really doesn't matter which one of these approaches you take.

In [22]:
import math

def median(numbers):
  numbers.sort()  # Sort the list of numbers
  count = len(numbers) # Get the length of the array
  isEven = count % 2 == 0 # Check if this list is of even length
  
  if (isEven):
    # Find the two numbers in the middle of the array
    mid = math.floor( count / 2 )
    a = numbers[mid - 1]
    b = numbers[mid]
    # Find the average of these two numbers
    ans = (a + b) / 2
  else:
    ans = numbers[math.floor( count / 2 )]
 
  return ans

print('The median of the charges is {}'.format(median(charges)))

The median of the charges is 3527.4049999999997


### Standard deviation

Recall that the standard deviation is calculated with the following formula:

$$ \sigma  = \sqrt\frac{\sum(X - \bar X)^2}{N - 1} $$

Where 
 * $X$: Each individual value
 * $\bar X$: The mean of all values
 * $\sum$: The sum of whatever comes next
 * $N$: The total number of values
 
You already have a function to calculate the mean, so go ahead and calculate the standard deviation:

In [23]:
def standard_deviation(numbers):
  X_bar = mean(numbers)
  N = len(numbers)
  total_sum = 0
  for X in numbers:
    diff = X - X_bar
    squared = math.pow(diff, 2)
    total_sum += squared
  sigma = math.sqrt(total_sum / (N - 1))  
  return sigma

print('The standard deviation of the charges is {}'.format(standard_deviation(charges)))

The standard deviation of the charges is 10114.063803685161


You can also leverage SQL queries to do the calculation for you. To do this, use the `STDDEV()` function around `charges`.

In [24]:
sql = '''
SELECT 
  STDDEV(charges)
FROM 
  medicalcosts
WHERE 
  sex = 'female'
  AND age BETWEEN 18 AND 30'''


charges = engine.execute(sql)
engine.dispose()

rows = charges.fetchall()[0][0] 

print('The standard deviation of the charges is {}'.format(rows))


The standard deviation of the charges is 10114.0612425204


## Another example

Your company would like to honor the hardest-working actor in comedy show business. They have access to a film database listing most of the popular movies released, along with some information about the actors involved. This database contains quite a lot of information, but you are mainly interested in the number of movies in which each actor appeared. 

The objective is to query this database for the list of actors and the number of comedy movies in which they appear. Then look for the actor with the highest number of appearances. You can also use the functions written above to get the mean, median, and standard deviation of these counts.

First, connect to the database.

In [25]:
# Database credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'dvdrentals'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))


Next, construct the SQL statement. You need the actor's name from the *actor* table, the category `Comedy` from the *category* table, and the film data from the *film* table. You need to join the film and actor using the *film_actor* table because there is a many-to-many relationship between films and actors. And similarly, you need to join the film to category using the *film_category* table.

Additionally, you don't need the list of movies and actors—you need the count. SQL has a built-in `COUNT()` function. To use it, first group the results by actor. Then perform a count within each group.

Finally, you can order the results by count in descending order so that you get the actor with most appearances first.

In [26]:
sql = '''
  SELECT COUNT(*) AS count, a.actor_id, a.first_name, a.last_name 
  FROM film f 
  JOIN film_actor fa 
  ON f.film_id = fa.film_id 
  JOIN actor a 
  ON a.actor_id = fa.actor_id 
  JOIN film_category fc 
  ON fc.film_id = f.film_id
  JOIN category c
  ON c.category_id = fc.category_id
  WHERE c.name = 'Comedy'
  GROUP BY a.actor_id
  ORDER BY count DESC
'''

results = engine.execute(sql)
engine.dispose()
rows = results.fetchall()

# Print some results just to see what you got
for row in rows:
  print(row)

(6, 196, 'BELA', 'WALKEN')
(5, 143, 'RIVER', 'DEAN')
(5, 149, 'RUSSELL', 'TEMPLE')
(4, 129, 'DARYL', 'CRAWFORD')
(4, 76, 'ANGELINA', 'ASTAIRE')
(4, 58, 'CHRISTIAN', 'AKROYD')
(4, 24, 'CAMERON', 'STREEP')
(4, 83, 'BEN', 'WILLIS')
(4, 37, 'VAL', 'BOLGER')
(4, 198, 'MARY', 'KEITEL')
(4, 101, 'SUSAN', 'DAVIS')
(4, 82, 'WOODY', 'JOLIE')
(4, 127, 'KEVIN', 'GARLAND')
(3, 159, 'LAURA', 'BRODY')
(3, 71, 'ADAM', 'GRANT')
(3, 162, 'OPRAH', 'KILMER')
(3, 84, 'JAMES', 'PITT')
(3, 173, 'ALAN', 'DREYFUSS')
(3, 42, 'TOM', 'MIRANDA')
(3, 125, 'ALBERT', 'NOLTE')
(3, 153, 'MINNIE', 'KILMER')
(3, 147, 'FAY', 'WINSLET')
(3, 19, 'BOB', 'FAWCETT')
(3, 81, 'SCARLETT', 'DAMON')
(3, 17, 'HELEN', 'VOIGHT')
(3, 54, 'PENELOPE', 'PINKETT')
(3, 34, 'AUDREY', 'OLIVIER')
(3, 90, 'SEAN', 'GUINESS')
(3, 45, 'REESE', 'KILMER')
(3, 107, 'GINA', 'DEGENERES')
(3, 89, 'CHARLIZE', 'DENCH')
(3, 31, 'SISSY', 'SOBIESKI')
(3, 158, 'VIVIEN', 'BASINGER')
(3, 13, 'UMA', 'WOOD')
(3, 133, 'RICHARD', 'PENN')
(3, 128, 'CATE', 'MCQUEEN')

You can see from this result that the actor Bela Walken appeared in six comedy movies. But, how much more than the other actors is that? Is it close to the average? To find out, you could extract the counts and get the mean and median:

In [27]:
counts = [row['count'] for row in rows]

print('The mean number of comedy movies by actor is {}'.format(mean(counts)))
print('The median number of comedy movies by actor is {}'.format(median(counts)))
print('The standard deviation of appearances in comedy movies is {}'.format(standard_deviation(counts)))

The mean number of comedy movies by actor is 1.945578231292517
The median number of comedy movies by actor is 2
The standard deviation of appearances in comedy movies is 1.051953908863022


You can draw a few quick conclusions from this result. First, a low standard deviation implies that the data points are close to the mean. That is, most actors appear in two comedy movies. Also, Bela Walken's six appearances is more than two standard deviations away from the mean, making it a statistically significant value. This actor is definitely working harder than the rest and deserving of your honor.

Of course, there is much more that can be done with this investigation before you make a conclusion like that.

## From databases

Sometimes, you'll want to get data from more than one database into pandas. You can use the information shown below to guide you on how to generate a DataFrame directly from the results of SQL queries. Using a SQL query to directly read into pandas will allow you to easily combine data from multiple tables.

In [ ]:
# Install the necessary PostgreSQL database adapter for Python (may not be necessary)
!pip install psycopg2-binary

import pandas as pd

# Database credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'medicalcosts'

# Use a database table to create a DataFrame
insurance_df = pd.read_sql_table('medicalcosts', 'postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

# Look at some details of the DataFrame
print(insurance_df)
insurance_df.head(2)

With this query, you can now think of the DataFrame as an in-memory representation of an entire database table. It's equally possible to create DataFrames from queries featuring filters or joins, like this:

In [ ]:
# Execute a query and create a DataFrame
females_df = pd.read_sql_query("SELECT age, bmi, smoker, region from medicalcosts WHERE sex = 'female'", 'postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

# Now, see some details of the DataFrame
print(females_df)
females_df.head(2)